# Coin Accounter System 

In [1]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from skimage.feature import local_binary_pattern
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeRegressor
from matplotlib import pyplot as plt
from os.path import isfile as exist
from sklearn.cluster import KMeans
from sklearn import preprocessing
from scipy.stats import itemfreq
from pylab import arange, array
from sklearn import metrics
import numpy as np
import cv2 as cv
import warnings
import pickle
import glob
import time

warnings.filterwarnings("ignore", category=DeprecationWarning)

In [2]:
# Função para facilitar a visualização das imagens
def showit(img, openCV=0, text='t'):
    
    if(openCV):
        cv.imshow(text, img)
        cv.waitKey(0)
        cv.destroyAllWindows()
    
    else:  
        # Para imagens preto e branco
        if(len(img.shape) < 3):
            plt.gray()
            plt.imshow(img)
            plt.show()
        else:
            plt.imshow(img[:,:,::-1])
            plt.show()

### Preprocessing Images

In [3]:
def resize(img, cols=800):    
    d = cols / img.shape[1]
    dim = (cols, int(img.shape[0] * d))
    img = cv.resize(img, dim, interpolation=cv.INTER_AREA)
    return img

def convolution(img, a, b):
    kernel = np.array([[a,a,a],[a,b,a],[a,a,a]])
    img = cv.filter2D(img,-1,kernel)
    
    return img

### Getting Coin ROI

In [10]:
singleCount = 0

# Função que retorna a mascara das moedas na imagem
def getMask(img):
    
    img = cv.bitwise_not(img, img)
    img = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    
    limiar, img = cv.threshold(img, 30, 255, cv.THRESH_BINARY) # limiar de otsu inclui as sombras
    
    stt = cv.getStructuringElement(cv.MORPH_ELLIPSE, (11,11))
    mask = cv.morphologyEx(img, cv.MORPH_CLOSE, stt, iterations=7)
    
    return mask

# Função que encontrar a moeda na imagem e retorna o menor quadrado envolvente com a moeda
def findSingleCoin(imgCol):
    
    # Reduz tamanho da imagem
    imgCol = resize(imgCol, 800)
    
    # Eliminar sombras da imagem
    imgCol = convolution(imgCol, -1/4, 4)
        
    # Encontro a Região de Interesse
    imgBin = getMask(imgCol)

    # Selecionando região de interesse na imagem colorida com a máscara encontrada
    imgCol = cv.bitwise_and(imgCol, imgCol, mask=imgBin)
    
    # Achar contornos das moedas
    moedas, hierarquia = cv.findContours(imgBin, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)
        
    # Para cada moeda encontrada
    for moeda in moedas:
        
        # Não contabilizar regiões de falhas
        area = cv.contourArea(moeda)
        if(area < 5000.0):            
            continue
            
        # Encontre o menor retângulo envolvente
        x, y, w, h = cv.boundingRect(moeda)

        # Nova imagem com o menor retângulo envolvente da moeda
        imgCoinRoi = imgCol[y:y+h, x:x+w]
        
        # Writing results in results folder
        global singleCount
        singleCount += 1
        cv.imwrite("singleCoins/coin{}.png".format(singleCount), imgCoinRoi)
        
        return imgCoinRoi

### Extractions Functions Routines

In [5]:
def saturationHistogram(img):
    img = cv.cvtColor(img, cv.COLOR_BGR2HSV)    
    hist = cv.calcHist([img],[0],None,[256],[0,256])
    hist = cv.normalize(hist, None)
    
    soma=0
    for i in hist.flatten():
        soma += i
    
    return soma/256

def coloredHistogram(img):
    
    histBlue = cv.calcHist([img[:,:,0]],[0],None,[256],[0,256])
    histBlue = cv.normalize(histBlue, None)
    
    histGreen = cv.calcHist([img[:,:,1]],[0],None,[256],[0,256])
    histGreen = cv.normalize(histGreen, None)
    
    histRed = cv.calcHist([img[:,:,2]],[0],None,[256],[0,256])
    histRed = cv.normalize(histRed, None)
    
    for i in range(len(histBlue)):
        histBlue[i] = min(histBlue[i], histGreen[i], histRed[i])
        
    soma=0
    for i in histBlue.flatten():
        soma += i
    
    return soma/256

In [6]:
def extraction(imag, roiDefined=False):
    
    if(not roiDefined):
        imag = findSingleCoin(imag)
        
    meanColor = coloredHistogram(imag)
    
    meanSaturation = saturationHistogram(imag)
    
    # LBP Invariant
    img = cv.cvtColor(imag.copy(), cv.COLOR_BGR2GRAY)
    lbp = local_binary_pattern(img,8,2,method='uniform')
    lbp_hist = itemfreq(lbp).flatten()
    lbp_hist = [float(i)/max(lbp_hist) for i in lbp_hist]
    
    return np.append(np.append(lbp_hist, meanColor), meanSaturation)
        
def extractionFromFile(file):
    img = cv.imread(file)
    return extraction(img)

## Set Train Dataset

In [11]:
start_time = time.time()

le = preprocessing.LabelEncoder()
le.fit(["Cinco", "Dez", "Vinte Cinco", "Cinquenta", "Real"])

if(exist("coin_features") and exist("coin_features_answer")):
    
    with open('coin_features', 'rb') as f:
        data = pickle.load(f)

    with open('coin_features_answer', 'rb') as f:
        answrData = pickle.load(f)
    
    with open('ArffData', 'rb') as f:
        arffData = pickle.load(f)
        
else:
    
    # Defina onde está a pasta do dataset da moeda
    coin_directory = "moedas/"

    # Lista com o nome das imagens nesse diretório
    sample_images_5_back = glob.glob(coin_directory + "5/back/*")
    sample_images_5_front = glob.glob(coin_directory + "5/front/*")

    sample_images_10_back = glob.glob(coin_directory  + "10/back/*")
    sample_images_10_front = glob.glob(coin_directory + "10/front/*")

    sample_images_25_back = glob.glob(coin_directory  + "25/back/*")
    sample_images_25_front = glob.glob(coin_directory + "25/front/*")

    sample_images_50_back = glob.glob(coin_directory  + "50/back/*")
    sample_images_50_front = glob.glob(coin_directory + "50/front/*")

    sample_images_100_back = glob.glob(coin_directory  + "100/back/*")
    sample_images_100_front = glob.glob(coin_directory + "100/front/*")

    # Lista com as características das imagens e suas respectivas classificações
    data = []
    answrData = []
    arffData = []
    
    qntFailure = 0
    FailureList = []
    
    # Extraindo características das moedas sozinhas
    for i in sample_images_5_front:
        try:
            features = extractionFromFile(i)
            data.append(features)
            answrData.append(le.transform(["Cinco"]))
            arffData.append((features, str(le.transform(["Cinco"]))))
        except:
            FailureList.append(i)
            qntFailure += 1
            continue


    for i in sample_images_5_back:
        try:
            features = extractionFromFile(i)
            data.append(features)
            answrData.append(le.transform(["Cinco"]))
            arffData.append((features, str(le.transform(["Cinco"]))))
        except:
            FailureList.append(i)
            qntFailure += 1
            continue

    for i in sample_images_10_front:
        try:
            features = extractionFromFile(i)
            data.append(features)
            answrData.append(le.transform(["Dez"]))
            arffData.append((features, str(le.transform(["Dez"]))))
        except:
            FailureList.append(i)
            qntFailure += 1
            continue

    for i in sample_images_10_back:
        try:
            features = extractionFromFile(i)
            data.append(features)
            answrData.append(le.transform(["Dez"]))
            arffData.append((features, str(le.transform(["Dez"]))))
        except:
            FailureList.append(i)
            qntFailure += 1
            continue

    for i in sample_images_25_front:
        try:
            features = extractionFromFile(i)
            data.append(features)
            answrData.append(le.transform(["Vinte Cinco"]))
            arffData.append((features, str(le.transform(["Vinte Cinco"]))))
        except:
            FailureList.append(i)
            qntFailure += 1
            continue

    for i in sample_images_25_back:
        try:
            features = extractionFromFile(i)
            data.append(features)
            answrData.append(le.transform(["Vinte Cinco"]))
            arffData.append((features, str(le.transform(["Vinte Cinco"]))))
        except:
            FailureList.append(i)
            qntFailure += 1
            continue

    for i in sample_images_50_front:
        try:
            features = extractionFromFile(i)
            data.append(features)
            answrData.append(le.transform(["Cinquenta"]))
            arffData.append((features, str(le.transform(["Cinquenta"]))))
        except:
            FailureList.append(i)
            qntFailure += 1
            continue

    for i in sample_images_50_back:
        try:
            features = extractionFromFile(i)
            data.append(features)
            answrData.append(le.transform(["Cinquenta"]))
            arffData.append((features, str(le.transform(["Cinquenta"]))))
        except:
            FailureList.append(i)
            qntFailure += 1
            continue

    for i in sample_images_100_front:
        try:
            features = extractionFromFile(i)
            data.append(features)
            answrData.append(le.transform(["Real"]))
            arffData.append((features, str(le.transform(["Real"])))) 
        except:
            FailureList.append(i)
            qntFailure += 1
            continue

    for i in sample_images_100_back:
        try:
            features = extractionFromFile(i)
            data.append(features)
            answrData.append(le.transform(["Real"]))
            arffData.append((features, str(le.transform(["Real"]))))
        except:
            FailureList.append(i)
            qntFailure += 1
            continue
    
    c = 0  
    for i in FailureList:
        img = cv.imread(i)
        c += 1
        cv.imwrite("fail_img/coin{}.png".format(c), img)
    
    print(qntFailure, "imagens falharam na extração de características")

    if(not qntFailure):
        # Save the results
        with open('coin_features', 'wb') as f:
            pickle.dump(data, f)

        with open('coin_features_answer', 'wb') as f:
            pickle.dump(answrData, f)

        with open('ArffData', 'wb') as f:
            pickle.dump(arffData, f)

print("--- %s seconds ---" % (time.time() - start_time))

0 imagens falharam na extração de características
--- 77.92105460166931 seconds ---


## Classifiers

In [12]:
# 75% para treino, 25% da medir a accuracia
data_train, data_test, asw_train, asw_test = train_test_split(
    data, answrData, test_size=.25)

In [13]:
trReg = DecisionTreeRegressor().fit(data_train,asw_train)
# t.predict(test)

score = int(trReg.score(data_test, asw_test) * 100)
print("Classifier mean accuracy: ", score, "%")
trReg = DecisionTreeRegressor().fit(data,answrData)

Classifier mean accuracy:  85 %


In [14]:
start_time = time.time()

mlp = MLPClassifier(solver="lbfgs").fit(data_train,asw_train)

score = int(mlp.score(data_test, asw_test) * 100)
print("Classifier mean accuracy: ", score, "%")
mlp = MLPClassifier(solver="lbfgs").fit(data,answrData)

print("--- %s seconds ---" % (time.time() - start_time))

/home/rogeriojunior/gpam/lib/python3.5/site-packages/sklearn/neural_network/multilayer_perceptron.py:916: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Classifier mean accuracy:  36 %


/home/rogeriojunior/gpam/lib/python3.5/site-packages/sklearn/neural_network/multilayer_perceptron.py:916: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


--- 0.7167706489562988 seconds ---


In [ ]:
clf = RandomForestClassifier(n_estimators=100).fit(data_train,asw_train)
# clf.predict(test)

score = int(clf.score(data_test, asw_test) * 100)
print("Classifier mean accuracy: ", score, "%")
clf = RandomForestClassifier(n_estimators=50, bootstrap = True, max_features = 'sqrt').fit(data_train,asw_train)

Classifier mean accuracy:  92 %


/home/rogeriojunior/gpam/lib/python3.5/site-packages/ipykernel_launcher.py:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """Entry point for launching an IPython kernel.
/home/rogeriojunior/gpam/lib/python3.5/site-packages/ipykernel_launcher.py:6: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


In [ ]:
tree = DecisionTreeClassifier().fit(data_train,asw_train)
# tree.predict()

score = int(tree.score(data_test, asw_test) * 100)
print("Classifier mean accuracy: ", score, "%")
tree = DecisionTreeClassifier().fit(data_train,asw_train).fit(data,answrData)

Classifier mean accuracy:  91 %


## Selecione qual classificador será usado

In [ ]:
# classifier = trReg  # DecisionTreeRegressor
# classifier = tree   # DecisionTreeClassifier
# classifier = mlp    # MLPClassifier
classifier = clf      # RandomForestClassifier

## Data Input

In [ ]:
def predictCoin(roi):
        
    hist = extraction(roi, True)
    s = classifier.predict([hist])
    
    # Lista com as probabilidade de cada moeda
    a = classifier.predict_proba([hist]).tolist()
        
    return le.inverse_transform(s.astype(int))[0], max(a[0])

def getMask(img, RoiDefined=False):

    img = cv.bitwise_not(img, img)
    
    img = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    limiar, img = cv.threshold(img, 0, 255, cv.THRESH_BINARY) # limiar de otsu inclui as sombras
        
    if(RoiDefined):
        size=2
        times=15
    else:
        size=5
        times=20
    
    stt = cv.getStructuringElement(cv.MORPH_ELLIPSE, (size,size))
    mask = cv.morphologyEx(img, cv.MORPH_CLOSE, stt, iterations=times)
        
#     showit(mask,1)
    
    return mask

def findMultiplesCoin(imgCol):
    
    imgBin = getMask(imgCol)
    imgCol = cv.bitwise_and(imgCol, imgCol, mask=imgBin)
        
    img = cv.cvtColor(imgCol, cv.COLOR_BGR2GRAY)
    img = cv.bilateralFilter(img, 5, 90, 90)
        
#     showit(img, 1)
    
    circles = cv.HoughCircles(img,method=cv.HOUGH_GRADIENT,dp=1,minDist=100,
                               param1=50,param2=50,minRadius=40,maxRadius=80)
    
    return circles

# Imagem que será analisada
ct = 0
def coin_detector(img, name):
    img = resize(img, 800)
    img = convolution(img, -1/4, 4)
    
    output = img.copy()
    
#     showit(img, 1)

    circles = findMultiplesCoin(img)

    form = img.copy()
    
    listCoins = []

    count = 0
    if circles is not None:
        
        # coordinates and radii
        circles = np.round(circles[0,:]).astype(int)

        for (x, y, d) in circles:

            count += 1 # Qnt de moedas
#             d+=7 # Increase radius

            form[:,:,:]=0

            cv.circle(form,(x,y), d, (255,255,255), -1)
            img2gray = cv.cvtColor(form, cv.COLOR_BGR2GRAY)
            ret, imgBin = cv.threshold(img2gray, 0, 255, cv.THRESH_BINARY)

#             showit(form,1)

            roi = cv.bitwise_and(img, img, mask=imgBin)
            roi = roi[y-d:y+d, x-d:x+d]
        
            listCoins.append(roi)

#             showit(roi[::2,::2],1)

            coin, chance = predictCoin(roi)

            chance = str(int(chance*100)) + " %"

            cv.circle(output, (x,y), d, (0, 255, 0), 2)
            cv.putText(output, coin, (x - 40, y), cv.FONT_HERSHEY_PLAIN,
                       1.5, (0, 255, 0), thickness=2, lineType=cv.LINE_AA)

            cv.putText(output, chance, (x - 20, y+30), cv.FONT_HERSHEY_PLAIN,
                       1.5, (0, 255, 0), thickness=2, lineType=cv.LINE_AA)
    
    showit(output, 1, name)
    global ct
    ct += 1
    cv.imwrite("final_result/coin{}.png".format(ct), output)
    
    return listCoins

In [ ]:
sample = glob.glob('moedas/juntas/*')

count = 0

for file in sample:
    img = cv.imread(file)
    
    listCoins = coin_detector(img, file)

## Arquivo ARFF para testar no Weka

In [ ]:
from os.path import isfile as exist

def gravar_arquivo_arff(base_teste, classes):
    tam = len(base_teste[0])
    file = open('data.arff','w') 
 
    file.write('@relation teste\n') 
    for i in range(tam):
        file.write('@attribute '+ str(i) +' NUMERIC\n') 
    
    file.write('@attribute classes {')
    
    a = set(classes)
    
    for i in a:
        file.write(str(i)+',')
    
    file.write('}')    
    for i in range(tam):
         len(set(classes))
    
    file.write('\n@data\n') 

    for item in base_teste:
        for i in range(len(item)):
            file.write("%s," % str(item[0][i])) 
        file.write("%s\n" % item[1])    
 
    file.close()
    
if(not exist("data.arff")):
    gravar_arquivo_arff(arffData,classes)